In [1]:
%pip install thefuzz

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from rapidfuzz import fuzz, process

In [3]:
df_retail = pd.read_csv("../data/retail-transactions/Retail_Transactions_Dataset.csv")
df_retail.head()

,Transaction_ID,Date,Customer_Name,Product,Total_Items,Total_Cost,Payment_Method,City,Store_Type,Discount_Applied,Customer_Category,Season,Promotion
0,1000000000,2022-01-21 06:27:29,Stacey Price,"['Ketchup', 'Shaving Cream', 'Light Bulbs']",3,71.65,Mobile Payment,Los Angeles,Warehouse Club,True,Homemaker,Winter,NaN
1,1000000001,2023-03-01 13:01:21,Michelle Carlson,"['Ice Cream', 'Milk', 'Olive Oil', 'Bread', 'P...",2,25.93,Cash,San Francisco,Specialty Store,True,Professional,Fall,BOGO (Buy One Get One)
2,1000000002,2024-03-21 15:37:04,Lisa Graves,['Spinach'],6,41.49,Credit Card,Houston,Department Store,True,Professional,Winter,NaN
3,1000000003,2020-10-31 09:59:47,Mrs. Patricia May,"['Tissues', 'Mustard']",1,39.34,Mobile Payment,Chicago,Pharmacy,True,Homemaker,Spring,NaN
4,1000000004,2020-12-10 00:59:59,Susan Mitchell,['Dish Soap'],10,16.42,Debit Card,Houston,Specialty Store,False,Young Adult,Winter,Discount on Selected Items


In [4]:
df_instacart_products = pd.read_csv("../data/instacart-market-basket-analysis/products.csv")
df_instacart_products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [5]:
df_instacart_orders = pd.read_csv("../data/instacart-market-basket-analysis/orders.csv")
df_instacart_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [6]:
df_instacart_order_products_prior = pd.read_csv("../data/instacart-market-basket-analysis/order_products_prior.csv")
df_instacart_order_products_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [7]:
df_instacart_order_products_train = pd.read_csv("../data/instacart-market-basket-analysis/order_products_train.csv")
df_instacart_order_products_train.head()

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


In [8]:
df_instacart_aisles = pd.read_csv("../data/instacart-market-basket-analysis/aisles.csv")
df_instacart_aisles.head()

,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation


In [9]:
df_instacart_departments = pd.read_csv("../data/instacart-market-basket-analysis/departments.csv")
df_instacart_departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [10]:
# Load the file with each row as a transaction
df_groceries = pd.read_csv("../data/groceries/groceries.csv", header=None, names=["Product_List"], on_bad_lines='skip', engine="python")

In [11]:
# Add Transaction IDs
df_groceries["Transaction_ID"] = range(1, len(df_groceries) + 1)

# Split the Product_List into individual items
df_groceries["Product_List"] = df_groceries["Product_List"].str.split(",")

# Explode into individual rows
grocery_transactions = df_groceries.explode("Product_List", ignore_index=True)

# Clean up whitespace
grocery_transactions["Product_List"] = grocery_transactions["Product_List"].str.strip()

# Remove NaN values
grocery_transactions.dropna(subset=["Product_List"], inplace=True)

# Rename column for clarity
grocery_transactions = grocery_transactions.rename(columns={"Product_List": "Product"})

# Display the final DataFrame
print(grocery_transactions)


                       Product  Transaction_ID
0                  ready soups               1
3                 meat spreads               4
4     long life bakery product               5
11                  newspapers              12
21                   chocolate              22
...                        ...             ...
6095                      soda            6096
6096                    pastry            6097
6097             bottled water            6098
6103             specialty bar            6104
6105              bottled beer            6106

[1005 rows x 2 columns]


In [12]:
df_store = pd.read_csv("../data/store-data/store_data.csv", header=None)

In [13]:
store_transactions = df_store.apply(lambda row: row.dropna().tolist(), axis=1).tolist()
store_transactions = [list(filter(None, transaction)) for transaction in store_transactions]

unique_store = pd.DataFrame({'Product' : store_transactions[0]})
unique_store = unique_store['Product'].unique()

print(unique_store)
print(len(unique_store))

['shrimp' 'almonds' 'avocado' 'vegetables mix' 'green grapes'
 'whole weat flour' 'yams' 'cottage cheese' 'energy drink' 'tomato juice'
 'low fat yogurt' 'green tea' 'honey' 'salad' 'mineral water' 'salmon'
 'antioxydant juice' 'frozen smoothie' 'spinach' 'olive oil']
20


In [14]:
df_retail['Product'] = df_retail['Product'].apply(eval)
retail_transactions = df_retail.explode('Product', ignore_index=True)

retail_transactions.head()

,Transaction_ID,Date,Customer_Name,Product,Total_Items,Total_Cost,Payment_Method,City,Store_Type,Discount_Applied,Customer_Category,Season,Promotion
0,1000000000,2022-01-21 06:27:29,Stacey Price,Ketchup,3,71.65,Mobile Payment,Los Angeles,Warehouse Club,True,Homemaker,Winter,NaN
1,1000000000,2022-01-21 06:27:29,Stacey Price,Shaving Cream,3,71.65,Mobile Payment,Los Angeles,Warehouse Club,True,Homemaker,Winter,NaN
2,1000000000,2022-01-21 06:27:29,Stacey Price,Light Bulbs,3,71.65,Mobile Payment,Los Angeles,Warehouse Club,True,Homemaker,Winter,NaN
3,1000000001,2023-03-01 13:01:21,Michelle Carlson,Ice Cream,2,25.93,Cash,San Francisco,Specialty Store,True,Professional,Fall,BOGO (Buy One Get One)
4,1000000001,2023-03-01 13:01:21,Michelle Carlson,Milk,2,25.93,Cash,San Francisco,Specialty Store,True,Professional,Fall,BOGO (Buy One Get One)


In [15]:
unique_retail = pd.DataFrame({'Product' : retail_transactions['Product'].unique()})
print(len(unique_retail))
print(unique_retail)

81
          Product
0         Ketchup
1   Shaving Cream
2     Light Bulbs
3       Ice Cream
4            Milk
..            ...
76    Canned Soup
77         Cheese
78            Jam
79        Diapers
80           Rice

[81 rows x 1 columns]


In [16]:
df_instacart_products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [17]:
unique_instacart_products = df_instacart_products['product_name']
print(len(unique_instacart_products))
print(unique_instacart_products)

49688
0                               Chocolate Sandwich Cookies
1                                         All-Seasons Salt
2                     Robust Golden Unsweetened Oolong Tea
3        Smart Ones Classic Favorites Mini Rigatoni Wit...
4                                Green Chile Anytime Sauce
                               ...                        
49683            Vodka, Triple Distilled, Twist of Vanilla
49684                   En Croute Roast Hazelnut Cranberry
49685                                     Artisan Baguette
49686           Smartblend Healthy Metabolism Dry Cat Food
49687                               Fresh Foaming Cleanser
Name: product_name, Length: 49688, dtype: object


In [22]:
aisle_dept_group = df_instacart_products.groupby(['aisle_id', 'department_id']).size()
# print(len(aisle_dept_group))
if (aisle_dept_group.sum() == len(df_instacart_products)):
    print('valid aisle & dept grouping')
    print(aisle_dept_group)
    aisle_dept_group.to_csv('aisle_dept_group.csv')
else:
    print('invalid aisle & dept grouping')

valid aisle & dept grouping
aisle_id  department_id
1         20               146
2         16               271
3         19               832
4         9                543
5         13               409
                          ... 
130       14               303
131       9                457
132       11               178
133       11               172
134       5                 95
Length: 134, dtype: int64


In [20]:
# Load the datasets
products = df_instacart_products
transactions = unique_retail

# Function to get the best match and its similarity score
def match_product(trans_product, product_list):
    match = process.extractOne(trans_product, product_list, scorer=fuzz.ratio)
    if match:  # Check if a match was found
        return match[0], match[1]  # Return the best match and the score
    return None, 0

# Create a product list from the products dataset
product_list = products['product_name'].tolist()

# Apply fuzzy matching to the `Product` column
transactions[['Best_Match', 'Similarity']] = transactions['Product'].apply(
    lambda x: pd.Series(match_product(x, product_list))
)

# Filter for strong matches if needed (e.g., similarity > 80%)
strong_matches = transactions[transactions['Similarity'] > 80]

# Save the results if needed
strong_matches.to_csv("fuzzy_matched_products.csv", index=False)

print(strong_matches.head())


     Product Best_Match  Similarity
0    Ketchup    Ketchup       100.0
3  Ice Cream  Ice Cream       100.0
4       Milk       Milk       100.0
5  Olive Oil  Olive Oil       100.0
6      Bread      Bread       100.0
